In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [2]:
# Load dataset
data = pd.read_csv('/content/auto-mpg.data', delim_whitespace=True, header=None, na_values='?')
data.columns = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin', 'car_name']
data = data.dropna()  # Drop rows with missing values

<ipython-input-2-9f754a276b28>:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv('/content/auto-mpg.data', delim_whitespace=True, header=None, na_values='?')


In [3]:
# Prepare features and target
X = data[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin']].values
y = data['mpg'].values  # Target variable is 'mpg'

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Create custom Dataset class for PyTorch
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [5]:
# Define the MLP model class
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_layers, activation_fn):
        super(MLPModel, self).__init__()
        layers = []

        for i, hidden_size in enumerate(hidden_layers):
            layers.append(nn.Linear(input_size if i == 0 else hidden_layers[i-1], hidden_size))
            if activation_fn == 'relu':
                layers.append(nn.ReLU())
            elif activation_fn == 'sigmoid':
                layers.append(nn.Sigmoid())
            elif activation_fn == 'tanh':
                layers.append(nn.Tanh())

        layers.append(nn.Linear(hidden_layers[-1], 1))  # Output layer for regression
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [6]:
# Define function to train and evaluate the model
def train_and_evaluate(hidden_layers, activation_fn, learning_rate, batch_size, epochs):
    # Initialize dataset and dataloaders
    train_dataset = CustomDataset(X_train, y_train)
    test_dataset = CustomDataset(X_test, y_test)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

    # Initialize model, loss function, and optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MLPModel(input_size=X_train.shape[1], hidden_layers=hidden_layers, activation_fn=activation_fn).to(device)
    criterion = nn.MSELoss()  # Mean Squared Error for regression
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Adam optimizer

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(features).squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}")  # Verbose output

    # Evaluate the model on the test set
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for features, labels in test_loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features).squeeze()
            loss = criterion(outputs, labels)
            total_loss += loss.item()

    avg_loss = total_loss / len(test_loader)
    return avg_loss

In [7]:
# Experiment 1: Comparing Hidden Layers and Neurons
hidden_layers_options = [[4], [8], [16], [32], [64], [128]]
results_hidden_layers = []
for hidden_layers in hidden_layers_options:
    avg_loss = train_and_evaluate(hidden_layers, activation_fn='relu', learning_rate=0.01, batch_size=32, epochs=50)
    results_hidden_layers.append({'hidden_layers': hidden_layers, 'avg_loss': avg_loss})

Epoch [1/50], Loss: 602.3215
Epoch [2/50], Loss: 593.0238
Epoch [3/50], Loss: 576.6078
Epoch [4/50], Loss: 557.5342
Epoch [5/50], Loss: 529.6365
Epoch [6/50], Loss: 496.5466
Epoch [7/50], Loss: 453.2599
Epoch [8/50], Loss: 399.6771
Epoch [9/50], Loss: 337.6751
Epoch [10/50], Loss: 267.0081
Epoch [11/50], Loss: 198.4614
Epoch [12/50], Loss: 139.4683
Epoch [13/50], Loss: 94.2440
Epoch [14/50], Loss: 68.0420
Epoch [15/50], Loss: 53.1720
Epoch [16/50], Loss: 45.8798
Epoch [17/50], Loss: 42.0495
Epoch [18/50], Loss: 38.7817
Epoch [19/50], Loss: 36.1266
Epoch [20/50], Loss: 33.2710
Epoch [21/50], Loss: 30.7407
Epoch [22/50], Loss: 28.3773
Epoch [23/50], Loss: 26.2974
Epoch [24/50], Loss: 24.4697
Epoch [25/50], Loss: 22.7032
Epoch [26/50], Loss: 21.4477
Epoch [27/50], Loss: 19.8935
Epoch [28/50], Loss: 18.8726
Epoch [29/50], Loss: 17.7221
Epoch [30/50], Loss: 16.8087
Epoch [31/50], Loss: 16.0793
Epoch [32/50], Loss: 15.5460
Epoch [33/50], Loss: 14.5577
Epoch [34/50], Loss: 13.8341
Epoch [35/5

In [8]:
# Experiment 2: Comparing Activation Functions
activation_functions = ['relu', 'sigmoid', 'tanh']
results_activation_functions = []
for activation_fn in activation_functions:
    avg_loss = train_and_evaluate(hidden_layers=[32], activation_fn=activation_fn, learning_rate=0.01, batch_size=32, epochs=50)
    results_activation_functions.append({'activation_function': activation_fn, 'avg_loss': avg_loss})

Epoch [1/50], Loss: 569.6687
Epoch [2/50], Loss: 469.4566
Epoch [3/50], Loss: 324.8467
Epoch [4/50], Loss: 165.2133
Epoch [5/50], Loss: 72.5006
Epoch [6/50], Loss: 58.6537
Epoch [7/50], Loss: 46.9060
Epoch [8/50], Loss: 34.4834
Epoch [9/50], Loss: 28.5074
Epoch [10/50], Loss: 23.6472
Epoch [11/50], Loss: 20.6384
Epoch [12/50], Loss: 18.3703
Epoch [13/50], Loss: 16.3573
Epoch [14/50], Loss: 15.2044
Epoch [15/50], Loss: 14.0456
Epoch [16/50], Loss: 13.2766
Epoch [17/50], Loss: 12.5357
Epoch [18/50], Loss: 12.1740
Epoch [19/50], Loss: 11.6121
Epoch [20/50], Loss: 11.1769
Epoch [21/50], Loss: 10.8482
Epoch [22/50], Loss: 10.4911
Epoch [23/50], Loss: 9.9290
Epoch [24/50], Loss: 10.1010
Epoch [25/50], Loss: 9.3393
Epoch [26/50], Loss: 9.1523
Epoch [27/50], Loss: 8.8849
Epoch [28/50], Loss: 8.9220
Epoch [29/50], Loss: 8.5871
Epoch [30/50], Loss: 8.4514
Epoch [31/50], Loss: 8.8513
Epoch [32/50], Loss: 8.1745
Epoch [33/50], Loss: 8.1649
Epoch [34/50], Loss: 8.1810
Epoch [35/50], Loss: 7.9600
Ep

In [9]:
# Experiment 3: Comparing Epochs
epochs_options = [1, 10, 25, 50, 100, 250]
results_epochs = []
for epochs in epochs_options:
    avg_loss = train_and_evaluate(hidden_layers=[32], activation_fn='relu', learning_rate=0.01, batch_size=32, epochs=epochs)
    results_epochs.append({'epochs': epochs, 'avg_loss': avg_loss})

Epoch [1/1], Loss: 582.3509
Epoch [1/10], Loss: 594.1282
Epoch [2/10], Loss: 507.7552
Epoch [3/10], Loss: 380.0792
Epoch [4/10], Loss: 215.2326
Epoch [5/10], Loss: 94.3584
Epoch [6/10], Loss: 55.0258
Epoch [7/10], Loss: 47.6670
Epoch [8/10], Loss: 34.2744
Epoch [9/10], Loss: 27.3889
Epoch [10/10], Loss: 22.5883
Epoch [1/25], Loss: 586.3988
Epoch [2/25], Loss: 488.5611
Epoch [3/25], Loss: 348.4142
Epoch [4/25], Loss: 190.5861
Epoch [5/25], Loss: 81.4382
Epoch [6/25], Loss: 50.6783
Epoch [7/25], Loss: 44.2249
Epoch [8/25], Loss: 32.4966
Epoch [9/25], Loss: 26.1709
Epoch [10/25], Loss: 21.6760
Epoch [11/25], Loss: 18.8227
Epoch [12/25], Loss: 16.5929
Epoch [13/25], Loss: 14.9596
Epoch [14/25], Loss: 13.8392
Epoch [15/25], Loss: 12.9900
Epoch [16/25], Loss: 12.6021
Epoch [17/25], Loss: 11.6503
Epoch [18/25], Loss: 11.0914
Epoch [19/25], Loss: 11.0200
Epoch [20/25], Loss: 10.4973
Epoch [21/25], Loss: 10.1117
Epoch [22/25], Loss: 9.8555
Epoch [23/25], Loss: 9.4880
Epoch [24/25], Loss: 9.0814

In [10]:
# Experiment 4: Comparing Learning Rates
learning_rates = [10, 1, 0.1, 0.01, 0.001, 0.0001]
results_learning_rates = []
for learning_rate in learning_rates:
    avg_loss = train_and_evaluate(hidden_layers=[32], activation_fn='relu', learning_rate=learning_rate, batch_size=32, epochs=50)
    results_learning_rates.append({'learning_rate': learning_rate, 'avg_loss': avg_loss})

Epoch [1/50], Loss: 9270424.2535
Epoch [2/50], Loss: 545491.4035
Epoch [3/50], Loss: 409122.9387
Epoch [4/50], Loss: 75096.6622
Epoch [5/50], Loss: 17888.2206
Epoch [6/50], Loss: 5291.9846
Epoch [7/50], Loss: 7132.9563
Epoch [8/50], Loss: 6117.2115
Epoch [9/50], Loss: 6229.7807
Epoch [10/50], Loss: 1903.3463
Epoch [11/50], Loss: 592.8818
Epoch [12/50], Loss: 244.4412
Epoch [13/50], Loss: 187.6109
Epoch [14/50], Loss: 168.1985
Epoch [15/50], Loss: 145.2602
Epoch [16/50], Loss: 120.4048
Epoch [17/50], Loss: 114.3137
Epoch [18/50], Loss: 93.9615
Epoch [19/50], Loss: 80.6497
Epoch [20/50], Loss: 78.2623
Epoch [21/50], Loss: 76.8053
Epoch [22/50], Loss: 79.3013
Epoch [23/50], Loss: 70.8064
Epoch [24/50], Loss: 62.3938
Epoch [25/50], Loss: 66.0133
Epoch [26/50], Loss: 73.9435
Epoch [27/50], Loss: 64.1644
Epoch [28/50], Loss: 60.7582
Epoch [29/50], Loss: 60.3020
Epoch [30/50], Loss: 67.3890
Epoch [31/50], Loss: 70.4997
Epoch [32/50], Loss: 60.3113
Epoch [33/50], Loss: 63.8902
Epoch [34/50], L

In [11]:
# Experiment 5: Comparing Batch Sizes
batch_sizes = [16, 32, 64, 128, 256, 512]
results_batch_sizes = []
for batch_size in batch_sizes:
    avg_loss = train_and_evaluate(hidden_layers=[32], activation_fn='relu', learning_rate=0.01, batch_size=batch_size, epochs=50)
    results_batch_sizes.append({'batch_size': batch_size, 'avg_loss': avg_loss})

Epoch [1/50], Loss: 542.5579
Epoch [2/50], Loss: 308.5729
Epoch [3/50], Loss: 84.8040
Epoch [4/50], Loss: 37.9960
Epoch [5/50], Loss: 23.5255
Epoch [6/50], Loss: 17.8796
Epoch [7/50], Loss: 14.8373
Epoch [8/50], Loss: 12.7664
Epoch [9/50], Loss: 12.1648
Epoch [10/50], Loss: 10.8269
Epoch [11/50], Loss: 10.6051
Epoch [12/50], Loss: 9.6594
Epoch [13/50], Loss: 9.2350
Epoch [14/50], Loss: 9.1053
Epoch [15/50], Loss: 8.4102
Epoch [16/50], Loss: 8.4390
Epoch [17/50], Loss: 8.0836
Epoch [18/50], Loss: 8.1038
Epoch [19/50], Loss: 8.0018
Epoch [20/50], Loss: 7.8582
Epoch [21/50], Loss: 7.6623
Epoch [22/50], Loss: 7.6817
Epoch [23/50], Loss: 7.6100
Epoch [24/50], Loss: 7.3919
Epoch [25/50], Loss: 7.3245
Epoch [26/50], Loss: 7.2832
Epoch [27/50], Loss: 7.3846
Epoch [28/50], Loss: 7.1158
Epoch [29/50], Loss: 7.0228
Epoch [30/50], Loss: 7.3979
Epoch [31/50], Loss: 7.5129
Epoch [32/50], Loss: 7.1068
Epoch [33/50], Loss: 7.1688
Epoch [34/50], Loss: 6.9342
Epoch [35/50], Loss: 6.7546
Epoch [36/50], L

In [12]:
# Consolidate and save results
hidden_layers_df = pd.DataFrame(results_hidden_layers)
activation_functions_df = pd.DataFrame(results_activation_functions)
epochs_df = pd.DataFrame(results_epochs)
learning_rates_df = pd.DataFrame(results_learning_rates)
batch_sizes_df = pd.DataFrame(results_batch_sizes)

# Save each result separately
hidden_layers_df.to_csv('mlp_hidden_layers_results_regression.csv', index=False)
activation_functions_df.to_csv('mlp_activation_functions_results_regression.csv', index=False)
epochs_df.to_csv('mlp_epochs_results_regression.csv', index=False)
learning_rates_df.to_csv('mlp_learning_rates_results_regression.csv', index=False)
batch_sizes_df.to_csv('mlp_batch_sizes_results_regression.csv', index=False)

print("Results saved for each experiment.")

Results saved for each experiment.
